In [1]:
import json
import jsonlines
import sys
import prompt_rag

search_engine = "brave"

with open(f"/home/hanlv/workspace/data/machine_learning/dataset/research/misinformation_dataset/COVMIS-main/data/train_{search_engine}_search.json", "r") as f:
    data_search = json.load(f)

try:
    with open(f"/home/hanlv/workspace/data/machine_learning/dataset/research/misinformation_dataset/COVMIS-main/data/train_{search_engine}_search_llm.json", "r") as f:
        data_search_llm = json.load(f)
except:
    data_search_llm = [{
        "claim": i["claim"],
        "claimant": i["claimant"],
        "label": i["label"],
        "date": i["date"],
    } for i in data_search]
    with open(f"/home/hanlv/workspace/data/machine_learning/dataset/research/misinformation_dataset/COVMIS-main/data/train_{search_engine}_search_llm.json", "w") as f:
        json.dump(data_search_llm, f, indent=4)


In [2]:
# prompt_rag.update_train_search_llm( 
#     "SOLAR-10.7B-Instruct-v1.0", "solar", port=8001, 
#     data_search=data_search, data_search_llm=data_search_llm) 



合并多个文件的先验知识

In [3]:
# # 对prior_knowledge做修改
# sort = False
# prior_knowledge_list = []

# K = 5
# prior_knowledge_version = "1"
# prior_knowledge_version_adjust = prior_knowledge_version + ".2"
# model_name = "mixtral"


# for i, item in enumerate(data_search_llm):
#     prior_knowledge = item[f"prior_knowledge_{model_name}_v{prior_knowledge_version}_K={K}"]
    
#     find_list = [
#         "Information Expansion and Summary:",
#         # "Information Summary:",
#         # "Expanded Information Summary:",
#         "Claim Analysis:",
#         # "Claim Verification",
#         # "Claim Analysis and Judgement:"
#     ]


#     for s in find_list:
#         pos = prior_knowledge.find(s)
#         if pos == -1:
#             continue
        
#         num_extra = 2
#         if pos!=0 and prior_knowledge[pos-1] == '\n':
#             pos -= 1
#             num_extra += 1
#         prior_knowledge = prior_knowledge[:pos] + prior_knowledge[pos + len(s) + num_extra :]

  
#     item[f"prior_knowledge_{model_name}_v{prior_knowledge_version_adjust}_K={K}"] = prior_knowledge


In [4]:
# bing search llm
# v2: k = 5
# v3: 不对时间排序

# brave search llm

# Solar
# v1: k = 5
# v2: k = 5 随机选取

# Mixtral
# v1: k = 5
# v1.1: 删除结构化内容（Information Expansion and Summary、Claim Analysis）

sort = False
prior_knowledge_list = []

K = 5
prior_knowledge_version = "1"
model_name = "mixtral"


# with open(f"train_search_llm_tmp.json", "r") as f:
#     prior_knowledge_list = json.load(f)

# for i, item in enumerate(data_search_llm):
#     if item["claim"] != prior_knowledge_list[i]["claim"].strip():
#         print(i)
#         print(item["claim"])
#         print(prior_knowledge_list[i]["claim"])
#         raise Exception()
#     else:
#         # vv = ""
#         # if prior_knowledge_list[i].get(f"prior_knowledge_{model_name}3") is not None:
#         #     vv = "3"
#         # elif prior_knowledge_list[i].get(f"prior_knowledge_{model_name}2") is not None:
#         #     vv = "2"

#         item[f"prior_knowledge_{model_name}_v{prior_knowledge_version}_K={K}"] = prior_knowledge_list[i][f"prior_knowledge_{model_name}"]

# data_search_llm[0]


In [5]:
# prompt_rag.save_search_llm(data_search_llm, search_engine)


创建数据（带有先验知识的Prompt）以微调LLM

In [6]:
# dict_list = []

# data_version = "2"
# for i, item in enumerate(data_search_llm):
    
#     if int(item["label"]) != 1:
#         prompt = prompt_rag.get_prompt_with_prior_knowledge(
#             item["claim"], 
#             search_engine,
#             data_search[i][f"{search_engine}_search_results"], 
#             item[f"prior_knowledge_{model_name}_v{prior_knowledge_version}_K={K}"], 
#             K=K,
#             claim_date=item["date"],
#             sort=sort,
#             known_info=False, # 不含已知信息
#             ids=None 
#         )
#         label = "TRUE." if int(item["label"]) == 2 else "FALSE."
#         dict_list.append({"query": prompt, "response": label})
# print(dict_list[0]["query"])
# len(dict_list), dict_list[:2]


In [7]:
# LLM_dir = "/home/hanlv/workspace/code/research/infodemic/LLM/"
# with jsonlines.open(
#     LLM_dir + \
#     f"swift/examples/pytorch/llm/my_data/with_{model_name}_info/{search_engine}/data{data_version}.jsonl", mode="w") as file_jsonl:
#     for line in dict_list:
#         file_jsonl.write(line)


测试模型的先验知识生成效果：一次提问

In [8]:
import torch
import os
import sys
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'

dirs = ["../.."]
for _dir in dirs:
    if _dir not in sys.path:
        sys.path.append(_dir)

from swift.llm import (
    ModelType, get_vllm_engine, get_default_template_type,
    get_template, inference_vllm, VllmGenerationConfig
)
from custom import CustomModelType, CustomTemplateType

# model_type = ModelType.mixtral_moe_7b_instruct
model_type = CustomModelType.llama_3_70b_instruct_gptq_int4
# model_type = CustomModelType.solar_instruct_10_7b

llm_engine = get_vllm_engine(
    model_type, 
    torch_dtype=torch.float16,  # 检查正确的数据类型！！！！
    tensor_parallel_size=2,
    max_model_len=4096,
    # gpu_memory_utilization=0.9,
    # model_id_or_path="/home/css/models/Mixtral-8x7B-Instruct-v0.1-GPTQ-int4",
    engine_kwargs = {
        # "enforce_eager": True,
        "max_num_seqs": 32,
        "seed": 42,
    }
)

template_type = get_default_template_type(model_type)
template = get_template(template_type, llm_engine.hf_tokenizer)

generation_config = VllmGenerationConfig(
    max_new_tokens=2048,
    temperature=0,
)

get_resp_list = lambda request_list : inference_vllm(
    llm_engine, template, request_list, 
    generation_config=generation_config, 
    use_tqdm=True,
)



2024-05-06 00:23:47,703 - modelscope - INFO - PyTorch version 2.2.1 Found.
2024-05-06 00:23:47,705 - modelscope - INFO - Loading ast index from /home/hanlv/.cache/modelscope/ast_indexer
2024-05-06 00:23:47,736 - modelscope - INFO - Loading done! Current index file version is 1.14.0, with md5 543353d790c7c1e5afd7a4dcd3609c68 and a total number of 976 components indexed
[INFO:swift] Loading the model using model_dir: /home/css/models/Meta-Llama-3-70B-Instruct-GPTQ-Int4
[INFO:swift] Setting torch_dtype: torch.float16
[INFO:swift] model_config: LlamaConfig {
  "_name_or_path": "/home/css/models/Meta-Llama-3-70B-Instruct-GPTQ-Int4",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128001,
  "hidden_act": "silu",
  "hidden_size": 8192,
  "initializer_range": 0.02,
  "intermediate_size": 28672,
  "max_position_embeddings": 8192,
  "model_type": "llama",
  "num_attention_heads": 64,
  "num_hidden

WARNING 05-06 00:23:48 config.py:169] gptq quantization is not fully optimized yet. The speed can be slower than non-quantized models.


2024-05-06 00:23:51,756	INFO worker.py:1749 -- Started a local Ray instance.


INFO 05-06 00:23:52 llm_engine.py:98] Initializing an LLM engine (v0.4.1) with config: model='/home/css/models/Meta-Llama-3-70B-Instruct-GPTQ-Int4', speculative_config=None, tokenizer='/home/css/models/Meta-Llama-3-70B-Instruct-GPTQ-Int4', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=auto, tensor_parallel_size=2, disable_custom_all_reduce=False, quantization=gptq, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=42)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 05-06 00:23:57 utils.py:608] Found nccl from library /home/hanlv/.config/vllm/nccl/cu12/libnccl.so.2.18.1
(RayWorkerWrapper pid=1051493) INFO 05-06 00:23:57 utils.py:608] Found nccl from library /home/hanlv/.config/vllm/nccl/cu12/libnccl.so.2.18.1
INFO 05-06 00:23:57 selector.py:77] Cannot use FlashAttention backend because the flash_attn package is not found. Please install it for better performance.
INFO 05-06 00:23:57 selector.py:33] Using XFormers backend.
(RayWorkerWrapper pid=1051493) INFO 05-06 00:23:58 selector.py:77] Cannot use FlashAttention backend because the flash_attn package is not found. Please install it for better performance.
(RayWorkerWrapper pid=1051493) INFO 05-06 00:23:58 selector.py:33] Using XFormers backend.
INFO 05-06 00:24:00 pynccl_utils.py:43] vLLM is using nccl==2.18.1
(RayWorkerWrapper pid=1051493) INFO 05-06 00:24:00 pynccl_utils.py:43] vLLM is using nccl==2.18.1
INFO 05-06 00:24:01 utils.py:129] reading GPU P2P access cache from /home/hanlv/.confi

In [44]:
s = "Information Summary:\n\nInformation 1, published on 2020-12-01, discusses a false claim about a Covid-19 vaccine having the ability to \"transfer genetic material\" and manipulate human genes. The claim was shared hundreds of times on Facebook, but experts, such as Dr. Kirsty Short, have confirmed that the mRNA vaccine cannot enter the human genome. Information 2, published on 2020-05-19, mentions false claims about a future Covid-19 vaccine genetically modifying humans. Information 3, with no specific publication date, states that Covid-19 vaccines do not change a person's genes and use messenger RNA or modified adenovirus to trigger an immune response. Information 4, published on 2023-11-01, reiterates that Covid-19 vaccines cannot alter a person's genome and debunks false claims about DNA contamination leading to harmful effects. Information 5, with no specific publication date, highlights the importance of monitoring and addressing vaccine misinformation to prevent vaccine hesitancy.\n\nRestated Claim:\nOn 2020-12-01, experts refuted the false claim that a Covid-19 vaccine can manipulate human genes.\n\nGiven the information available, the claim is TRUE. The Covid-19 vaccines developed by Pfizer-BioNTech, Moderna, and Johnson & Johnson do not have the ability to manipulate human genes, as confirmed by various experts and sources. These vaccines either use messenger RNA or modified adenovirus to trigger an immune response, but they cannot alter human DNA."

print(s)


Information Summary:

Information 1, published on 2020-12-01, discusses a false claim about a Covid-19 vaccine having the ability to "transfer genetic material" and manipulate human genes. The claim was shared hundreds of times on Facebook, but experts, such as Dr. Kirsty Short, have confirmed that the mRNA vaccine cannot enter the human genome. Information 2, published on 2020-05-19, mentions false claims about a future Covid-19 vaccine genetically modifying humans. Information 3, with no specific publication date, states that Covid-19 vaccines do not change a person's genes and use messenger RNA or modified adenovirus to trigger an immune response. Information 4, published on 2023-11-01, reiterates that Covid-19 vaccines cannot alter a person's genome and debunks false claims about DNA contamination leading to harmful effects. Information 5, with no specific publication date, highlights the importance of monitoring and addressing vaccine misinformation to prevent vaccine hesitancy.



In [9]:
def get_prompt_for_generating_prior_knowledge(
        claim, claim_date, search_engine, search_results, model_name,
        K=5, sort=False, ids=None, without_info=False, without_claim_date=False):
    """
    sort: 对search result 按时间进行排序
    """

    claim = claim.strip()

    if model_name == "solar":
        pre = "Below is some INFORMATION searched online and a CLAIM. These pieces of INFORMATION are relevant to the CLAIM. This CLAIM and all INFORMATION include their respective publication dates and contents. To classify the CLAIM more accurately (if the content described by the CLAIM is correct, it will be classified as TRUE; if the content described by the CLAIM is incorrect, it will be classified as FALSE), please first expand on the given INFORMATION and provide a detailed summary of it. Then analyze, reason, and provide reasonable evidence to judge the correctness of the CLAIM based on the available information and your knowledge, and finally generate prior knowledge that helps classify the CLAIM.\n\n"
    elif model_name == "mixtral":
        # v1
        # pre = "Below is a CLAIM and some INFORMATION searched online. These pieces of INFORMATION are relevant to the CLAIM. This CLAIM and all INFORMATION include their respective publication dates and contents. To classify the CLAIM more accurately (if the content described by the CLAIM is correct, it will be classified as TRUE; if the content described by the CLAIM is incorrect, it will be classified as FALSE), please first provide a detailed summary of the given INFORMATION and restate the CLAIM. Then reason, and provide reasonable evidence to judge the correctness of the CLAIM based on the available information and your knowledge. In reasoning, it is necessary to consider the sequential relationship between the date of publication of the CLAIM and the date of publication of the INFORMATION.\n\n"
        # v2
        # pre = "Below is some INFORMATION searched online and a CLAIM. These pieces of INFORMATION are relevant to the CLAIM. This CLAIM and all INFORMATION include their respective publication dates and contents. To classify the CLAIM more accurately (if the content described by the CLAIM is correct, it will be classified as TRUE; if the content described by the CLAIM is incorrect, it will be classified as FALSE), please first provide a detailed summary of the given INFORMATION and restate the CLAIM. Then reason, and provide reasonable evidence to judge the correctness of the CLAIM based on the available information and your knowledge.\n\n"
        
        pre = "Below is some INFORMATION searched online and a CLAIM. These pieces of INFORMATION are relevant to the CLAIM. This CLAIM and all INFORMATION include their respective publication dates and contents. To classify the CLAIM more accurately (TRUE or FALSE), please first provide a clear summary of the given INFORMATION, restate the CLAIM, and provide reasonable evidence to judge the correctness of the CLAIM based on the available information and your knowledge."

    else:
        raise Exception("model_name 只能从solar，mixtral中选择")
    
    if without_claim_date:
        text = "CLAIM: " + claim
    else:
        text = "CLAIM: " + get_claim_with_date(claim, claim_date)

    if search_engine == "bing":
        snippet = prompt_rag.get_bing_snippet_v2(search_results, K=K, claim_date=claim_date, sort=sort)
    elif search_engine == "brave":
        if ids is None:
            ids = slice(0, K)
        snippet = prompt_rag.get_brave_snippet(search_results, ids=ids)
    else:
        raise Exception("Select search engines in [\"bing\", \"brave\"].")
    
    info = "INFORMATION:\n" + snippet + '\n'

    if without_info:
        return (pre + text).strip()
    else:
        return pre + info + text

def get_claim_with_date(claim, claim_date=None):
    if claim_date is None:
        return " " + claim
    
    # res = "\n"
    res = claim + "\nPublication date: " + claim_date
    return res

K = 5
def get_id(claim):
    for i in range(len(data_search)):
        if claim.strip() in data_search[i]["claim"].strip():
            return i

# i = 0

i = get_id("A 2010 study on vaccines did not show that one in forty were damaged by vaccination")

claim = data_search[i]["claim"]
search_results = data_search[i][f"{search_engine}_search_results"]

model_name = 'mixtral'
prompt_list1 = [
    get_prompt_for_generating_prior_knowledge(
        claim, data_search[i]["date"], search_engine, search_results, model_name,
        K=K, sort=False, 
        # ids=data_search[i]["random_ids"],
        ids=None
    ),
]

request_list1 = [{'query': prompt} for prompt in prompt_list1]

print(prompt_list1[0])
print()

resp_list1 = get_resp_list(request_list1)
print(resp_list1[0]["response"].strip())
print()
resp_list1[0]["response"].strip()


Below is some INFORMATION searched online and a CLAIM. These pieces of INFORMATION are relevant to the CLAIM. This CLAIM and all INFORMATION include their respective publication dates and contents. To classify the CLAIM more accurately (if the content described by the CLAIM is correct, it will be classified as TRUE; if the content described by the CLAIM is incorrect, it will be classified as FALSE), please first provide a detailed summary of the given INFORMATION and restate the CLAIM. Then reason, and provide reasonable evidence to judge the correctness of the CLAIM based on the available information and your knowledge.

INFORMATION:
Information 1:
Publication date: 2020-12-16
Title: Fact check: A 2010 study on vaccines did not show that one in forty ...
Content:
A video posted on Facebook by an anti-lockdown activist, citing a 2010 vaccination study, claims that one in forty have “vaccine damage.” A doctor who worked on the study said that the claim in the video misinterpreted the st

100%|██████████| 1/1 [00:36<00:00, 36.77s/it]

**Summary of INFORMATION**

The provided information consists of five pieces of content related to vaccines and their safety. Here's a brief summary of each:

1. A fact-checking article from 2020 debunks a claim that a 2010 study found that one in forty people experience "vaccine damage." The article states that the study actually counted medical events detected within 30 days of vaccination, without inferring causal links.
2. A report (no publication date provided) highlights the global impact of COVID-19 vaccination, stating that it has saved tens of millions of lives, but notes that inadequate access to vaccines in low-income countries has limited its impact.
3. A webpage from the Centers for Disease Control and Prevention (CDC) discusses historical safety concerns related to vaccines, including a 2014 study on the association between H1N1 influenza vaccines and narcolepsy, and the discovery of porcine circovirus (PCV) in rotavirus vaccines.
4. A CDC webpage from 2020 reports on rar

'**Summary of INFORMATION**\n\nThe provided information consists of five pieces of content related to vaccines and their safety. Here\'s a brief summary of each:\n\n1. A fact-checking article from 2020 debunks a claim that a 2010 study found that one in forty people experience "vaccine damage." The article states that the study actually counted medical events detected within 30 days of vaccination, without inferring causal links.\n2. A report (no publication date provided) highlights the global impact of COVID-19 vaccination, stating that it has saved tens of millions of lives, but notes that inadequate access to vaccines in low-income countries has limited its impact.\n3. A webpage from the Centers for Disease Control and Prevention (CDC) discusses historical safety concerns related to vaccines, including a 2014 study on the association between H1N1 influenza vaccines and narcolepsy, and the discovery of porcine circovirus (PCV) in rotavirus vaccines.\n4. A CDC webpage from 2020 repor

In [10]:
def get_prompt_for_generating_prior_knowledge2(
        claim, claim_date, search_engine, search_results, model_name,
        K=5, sort=False, ids=None, without_info=False, without_claim_date=False):
    """
    sort: 对search result 按时间进行排序
    """

    claim = claim.strip()

    if model_name == "solar":
        pre = "Below is some INFORMATION searched online and a CLAIM. These pieces of INFORMATION are relevant to the CLAIM. This CLAIM and all INFORMATION include their respective publication dates and contents. To classify the CLAIM more accurately (if the content described by the CLAIM is correct, it will be classified as TRUE; if the content described by the CLAIM is incorrect, it will be classified as FALSE), please first expand on the given INFORMATION and provide a detailed summary of it. Then analyze, reason, and provide reasonable evidence to judge the correctness of the CLAIM based on the available information and your knowledge, and finally generate prior knowledge that helps classify the CLAIM.\n\n"
    elif model_name == "mixtral":
        # pre = "Below is a CLAIM and some INFORMATION searched online. These pieces of INFORMATION are relevant to the CLAIM. This CLAIM and all INFORMATION include their respective publication dates and contents. To classify the CLAIM more accurately (if the content described by the CLAIM is correct, it will be classified as TRUE; if the content described by the CLAIM is incorrect, it will be classified as FALSE), please first provide a detailed summary of the given INFORMATION and restate the CLAIM. Then reason, and provide reasonable evidence to judge the correctness of the CLAIM based on the available information and your knowledge. In reasoning, it is necessary to consider the sequential relationship between the date of publication of the CLAIM and the date of publication of the INFORMATION.\n\n"
        pre = "Below is some INFORMATION searched online and a CLAIM. These pieces of INFORMATION are relevant to the CLAIM. This CLAIM and all INFORMATION include their respective publication dates and contents. To classify the CLAIM more accurately (if the content described by the CLAIM is correct, it will be classified as TRUE; if the content described by the CLAIM is incorrect, it will be classified as FALSE), please first provide a detailed summary of the given INFORMATION and restate the CLAIM. Then reason, and provide reasonable evidence to judge the correctness of the CLAIM based on the available information and your knowledge. In reasoning, it is necessary to consider the sequential relationship between the date of publication of the CLAIM and the date of publication of the INFORMATION.\n\n"
    else:
        raise Exception("model_name 只能从solar，mixtral中选择")
    
    if without_claim_date:
        text = "CLAIM: " + claim
    else:
        text = "CLAIM:" + get_claim_with_date(claim, claim_date)

    if search_engine == "bing":
        snippet = prompt_rag.get_bing_snippet_v2(search_results, K=K, claim_date=claim_date, sort=sort)
    elif search_engine == "brave":
        if ids is None:
            ids = slice(0, K)
        snippet = prompt_rag.get_brave_snippet(search_results, ids=ids)
    else:
        raise Exception("Select search engines in [\"bing\", \"brave\"].")
    
    info = "INFORMATION:\n" + snippet + '\n'

    if without_info:
        return (pre + text).strip()
    else:
        return pre + info + text

def get_prompt_for_generating_prior_knowledge1(
        claim, claim_date, search_engine, search_results, model_name,
        K=5, sort=False, ids=None, without_info=False, without_claim_date=False):
    """
    sort: 对search result 按时间进行排序
    """

    claim = claim.strip()

    if model_name == "solar":
        pre = "Below is some INFORMATION searched online and a CLAIM. These pieces of INFORMATION are relevant to the CLAIM. This CLAIM and all INFORMATION include their respective publication dates and contents. To classify the CLAIM more accurately (if the content described by the CLAIM is correct, it will be classified as TRUE; if the content described by the CLAIM is incorrect, it will be classified as FALSE), please first expand on the given INFORMATION and provide a detailed summary of it. Then analyze, reason, and provide reasonable evidence to judge the correctness of the CLAIM based on the available information and your knowledge, and finally generate prior knowledge that helps classify the CLAIM.\n\n"
    elif model_name == "mixtral":
        # pre = "Below is a CLAIM and some INFORMATION searched online. These pieces of INFORMATION are relevant to the CLAIM. This CLAIM and all INFORMATION include their respective publication dates and contents. To classify the CLAIM more accurately (if the content described by the CLAIM is correct, it will be classified as TRUE; if the content described by the CLAIM is incorrect, it will be classified as FALSE), please first provide a detailed summary of the given INFORMATION and restate the CLAIM. Then reason, and provide reasonable evidence to judge the correctness of the CLAIM based on the available information and your knowledge. In reasoning, it is necessary to consider the sequential relationship between the date of publication of the CLAIM and the date of publication of the INFORMATION.\n\n"
        pre = "Below is a CLAIM and some INFORMATION searched online. These pieces of INFORMATION are relevant to the CLAIM. This CLAIM and all INFORMATION include their respective publication dates and contents. To classify the CLAIM more accurately (if the content described by the CLAIM is correct, it will be classified as TRUE; if the content described by the CLAIM is incorrect, it will be classified as FALSE), please first provide a detailed summary of the given INFORMATION and restate the CLAIM. Then reason, and provide reasonable evidence to judge the correctness of the CLAIM based on the available information and your knowledge. In reasoning, it is necessary to consider the sequential relationship between the date of publication of the CLAIM and the date of publication of the INFORMATION.\n\n"
    else:
        raise Exception("model_name 只能从solar，mixtral中选择")
    
    if without_claim_date:
        text = "CLAIM: " + claim
    else:
        text = "CLAIM: " + get_claim_with_date(claim, claim_date) + '\n\n'

    if search_engine == "bing":
        snippet = prompt_rag.get_bing_snippet_v2(search_results, K=K, claim_date=claim_date, sort=sort)
    elif search_engine == "brave":
        if ids is None:
            ids = slice(0, K)
        snippet = prompt_rag.get_brave_snippet(search_results, ids=ids)
    else:
        raise Exception("Select search engines in [\"bing\", \"brave\"].")
    
    info = "INFORMATION:\n" + snippet

    if without_info:
        return (pre + text).strip()
    else:
        return pre + text + info
    
def get_claim_with_date(claim, claim_date=None):
    if claim_date is None:
        return " " + claim
    
    # res = "\n"
    res = claim + "\nPublication date: " + claim_date
    return res

K = 5
def get_id(claim):
    for i in range(len(data_search)):
        if claim.strip() in data_search[i]["claim"].strip():
            return i

# i = 0

i = get_id("Pfizer and Moderna do call their COVID-19 shot a ‘vaccine’")
ids = [
    get_id("Pfizer and Moderna do call their COVID-19 shot a ‘vaccine’"),
    get_id("Myth spreads online that Australian supermarkets have banned Chinese nationals during COVID-19 pandemic"),
    get_id("Trump makes false claims about COVID-19 testing"),
    get_id("UNHCR condemns fake notice which claimed refugees in Malaysia are resisting COVID-19 tests"),
    get_id("Hospitals in Ohio were not set on fire during protests"),
    get_id("A 2010 study on vaccines did not show that one in forty were damaged by vaccination"),
    get_id("The Philippine health department said it did not issue this 'checklist' for COVID-19 symptoms"),
    get_id("Contaminated CDC COVID-19 test kits recalled and did not spread virus"),
    get_id("Experts refute false claim that Covid-19 vaccine can 'manipulate' human genes"),
    get_id("Countries were not buying Covid-19 test kits in 2018"),
    get_id("False claim circulates that Pakistani plane transported Sri Lankan students home after COVID-19 lockdown"),
    get_id("People will not have to be vaccinated against COVID-19 to receive food stamps and rent assistance"),
    get_id("Britain has not awarded a contract to develop a vaccine passport"),
    get_id("World Health Organization says COVID-19 means ‘coronavirus disease 2019’ – not 'China outbreak virus'"),
    get_id("The common cold is not the same as COVID-19 and the NHS is not saying it is"),
    get_id("Philippine authorities did not issue this warning after the novel coronavirus outbreak"),
    get_id("No tourists have been allowed to visit New Zealand since March 2020 -- this photo has circulated online since 2016"),
    get_id("This video does not show social distancing failure on an Air India flight during the coronavirus pandemic"),
    get_id("This photo shows a Sri Lankan airline pilot who tested positive for the novel coronavirus."), # fALSE
]

prompt_list = []
model_name = 'mixtral'
for i in ids:
    claim = data_search[i]["claim"]
    search_results = data_search[i][f"{search_engine}_search_results"]
    prompt_list.append(get_prompt_for_generating_prior_knowledge(
            claim, data_search[i]["date"], search_engine, search_results, model_name,
            K=K, sort=False, 
            # ids=data_search[i]["random_ids"],
            ids=None
        ))
request_list = [{'query': prompt} for prompt in prompt_list]

resp_list = get_resp_list(request_list)


100%|██████████| 19/19 [05:13<00:00, 16.50s/it]


In [23]:
# info + CLAIM wrong: 

print(len(ids))


nn = 10
print(prompt_list[nn])
print('*'*50)
print(resp_list[nn]["response"].strip())


19
Below is some INFORMATION searched online and a CLAIM. These pieces of INFORMATION are relevant to the CLAIM. This CLAIM and all INFORMATION include their respective publication dates and contents. To classify the CLAIM more accurately (if the content described by the CLAIM is correct, it will be classified as TRUE; if the content described by the CLAIM is incorrect, it will be classified as FALSE), please first provide a detailed summary of the given INFORMATION and restate the CLAIM. Then reason, and provide reasonable evidence to judge the correctness of the CLAIM based on the available information and your knowledge.

INFORMATION:
Information 1:
Publication date: 2020-04-28
Title: False claim circulates that Pakistani plane transported Sri Lankan ...
Content:
Sri Lakan airline aircrafts are seen at the apron of the Bandaranaike International airport in Katunayake on March 19, 2020, following the Sri Lankan government order to ban all incoming flights for two weeks as preventive 